In [11]:
import csv
import pandas as pd
import numpy as np
import warnings
from collections import Counter
from sklearn.naive_bayes import MultinomialNB      
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

warnings.filterwarnings('ignore')

In [2]:
#Lendo o arquivo csv com o pandas
df = pd.read_csv('./dados/site_alura.csv', encoding="utf-8")

In [3]:
"""
Separando as colunas do csv entre:
X = Características do dados
Y = Classificação das características 
"""

X = df[["recencia", "frequencia", "semanas"]]
Y = df["situacao"]

In [ ]:
X_dummies = pd.get_dummies(X)
X = X_dummies.values

In [25]:
porcentagem_treino = 0.8

tamanho_treino = int(porcentagem_treino * len(X))

treino_dados = X[0:tamanho_treino]
treino_marcacoes = Y[0: tamanho_treino]

validacao_dados = X[tamanho_treino:]
validacao_marcacoes = Y[tamanho_treino:]

In [5]:
print("Tamanho de treino: ", tamanho_treino)
print("Tamanho de validação: ", tamanho_validacao)

Tamanho de treino:  180
Tamanho de validação:  45


In [21]:
"""Verificando a acuracia do modelo, comparando com modelos burros"""

contador = Counter(validacao_marcacoes)
taxa_acerto_base = 100.0 * max(Counter(validacao_marcacoes).values())/len(validacao_marcacoes)

print("Contador: ", contador)
print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

Contador:  Counter({1: 34, 0: 6, 2: 5})
Taxa de acertos base: 75.56


In [30]:
def fit_and_predict(modelo, nome_algoritmo, treino_dados, treino_marcacoes):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        cross_val_score faz a quantidade de predições que forem colocadas no K e depois tira a média, é feito esse
        tipo de abordagem para garantir que o resultado da predição não esta viciada devido ao posicionamento dos dados.
    """
    k = 10

    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)

    taxa_media_acerto = np.mean(scores)
                    
    print("Média da taxa de acertos para o algoritmo {}, é de: {:.2f}" .format(nome_algoritmo, taxa_media_acerto))
    return taxa_media_acerto

In [23]:
def fit_and_predict_dados_reais(modelo, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        resultado = São as inferências que o modelo predice tendo por base as caracteristicas dos dados passados 
    """
    modelo.fit(treino_dados, treino_marcacoes)
    
    resultado = modelo.predict(validacao_dados)

    diferencas = (resultado == validacao_marcacoes)

    acertos = sum(diferencas)
    print("Taxa de acertos para algoritmo real, é de: {:.2f}" .format(100 * (acertos / len(validacao_dados))))

In [31]:
"""
Criando quatro modelos: MultinomialNB, AdaBoostClassifier, OneVersusRest, OneVersusOne

Obs.: De maneira geral a previsão dos algoritmos funcionam identificando e classificando os elementos por meio
das características que foram passadas no treinamento.
Obs.1: Se a performance não for problema o algoritmo OneVersusOne deve ser executado.
"""
resultados = {}
print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

modelo_multinomial = MultinomialNB()
resultado_multinomial = fit_and_predict(modelo_multinomial, "MultinomialNB", treino_dados, treino_marcacoes)
resultados[resultado_multinomial] = modelo_multinomial

modelo_adaBoostClassifier = AdaBoostClassifier()
resultado_adaboost = fit_and_predict(modelo_adaBoostClassifier, "AdaBoostClassifier", treino_dados, treino_marcacoes)
resultados[resultado_adaboost] = modelo_adaBoostClassifier

modelo_oneVersusRest = OneVsRestClassifier(LinearSVC(random_state=0))
resultado_oneVersusRest = fit_and_predict(modelo_oneVersusRest, "OneVersusRest", treino_dados, treino_marcacoes)                                         
resultados[resultado_oneVersusRest] = modelo_oneVersusRest

modelo_OneVersusOne = OneVsOneClassifier(LinearSVC(random_state=0))
resultado_OneVersusOne = fit_and_predict(modelo_OneVersusOne, "OneVersusOne", treino_dados, treino_marcacoes)
resultados[resultado_OneVersusOne] = modelo_OneVersusOne

vencedor = resultados[max(resultados)]
fit_and_predict_dados_reais(vencedor, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes)

Taxa de acertos base: 75.56
Média da taxa de acertos para o algoritmo MultinomialNB, é de: 0.83
Média da taxa de acertos para o algoritmo AdaBoostClassifier, é de: 0.76
Média da taxa de acertos para o algoritmo OneVersusRest, é de: 0.93
Média da taxa de acertos para o algoritmo OneVersusOne, é de: 0.99
Taxa de acertos para algoritmo real, é de: 100.00
